# 基于LLM微调的数学推理任务

In [1]:
import os 
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from datasets import load_dataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
# # 设置代理
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [3]:
from datasets import load_dataset



test_dataset = load_dataset('json', data_files='../MATH_test.jsonl')
train_dataset = load_dataset('json', data_files='../MATH_train.jsonl', split='train')



Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
def get_dataloader(prompt, batch_size=1):
    def preprocess(dataset, shuffle=True):
        def collate_fn(batch):
            questions =  [[{"role": "user", "content": prompt.format(text=item['problem'])}] for item in batch]
            answers = [item["solution"] for item in batch]
            return questions, answers
        return DataLoader(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn,
        )
    testset = test_dataset
    return preprocess(testset, shuffle=False)

In [6]:
 ## setup math_equivalence ： 在math工作目录中 pip install .
import math_equivalence
def eval_process(model_name, tokenizer_name, prompt, batch_size=1, ratio=1):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        cache_dir = '/autodl-tmp/cache'
    )
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    testloader = get_dataloader(prompt, batch_size)
    answers = []
    replies = []
    # 以下过程参照huggingface上的qwen2.5模型示例
    for idx, (texts, truths) in tqdm(enumerate(testloader), total=int(len(testloader) * ratio)):
        if idx >= int(len(testloader) * ratio):
            break
        texts = [tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        ) for text in texts]
        model_inputs = tokenizer(texts, return_tensors="pt", padding=True, padding_side='left').to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=256
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        replies.extend(responses)
        answers.extend(truths)
    
    total = 0
    correct = 0
    for reply, answer in zip(replies, answers):
        total += 1
        correct += math_equivalence.is_equiv(reply, answer)
    print(f"正确率: {(correct / total) * 100: .2f}%")

In [10]:
model_name = "output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-450"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct"

prompt = '''You are a highly skilled mathematician capable of solving complex grade-school math problems step by step. Please read the problem carefully and solve it with clear reasoning. Follow these instructions:

- Identify the known information and the question being asked.
- Break the solution into logical steps, providing clear explanations for each.
- Show all calculations and intermediate results.
- Conclude with a final answer.

Output your response in the following format:
[Explanation and calculations]
#### [Final numerical answer]   

Here is the problem:
{text}'''
eval_process(model_name, tokenizer_name, prompt, batch_size=8, ratio=1)

OSError: Incorrect path_or_model_id: 'output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-450'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [11]:
# 导入swift框架进行微调
from swift.llm import (
    DatasetName, InferArguments, ModelType, SftArguments,
    infer_main, sft_main, app_ui_main
)

ModuleNotFoundError: No module named 'swift'

In [8]:
model_type = ModelType.qwen2_5_0_5b_instruct
#TODO : change the dataset
sft_args = SftArguments(
    model_type=model_type,
    dataset=['MATH_train_fineture.jsonl'],
    output_dir='output',
    max_length=4096)
result = sft_main(sft_args)
last_model_checkpoint = result['last_model_checkpoint']

NameError: name 'ModelType' is not defined